# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray`

In [36]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [37]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [38]:
# confirm that our new database was created
print(mongo.list_database_names())

["'gardenDB'", 'admin', 'classDB', 'config', 'fruits_db', 'local', 'maggie_db', 'met', 'travel_db', 'uk_food']


In [39]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [40]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [41]:
# review a document in the establishments collection
print(db.customer_list.find_one())

None


In [42]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [43]:
# Create a dictionary for the new restaurant data
restaurants = {}

In [44]:
# Insert the new restaurant into the collection
restaurants = {"name" : "Penang Flavours"}

establishments.insert_one(restaurants)

InsertOneResult(ObjectId('65fcc09ee2745ce6ea87e1fd'), acknowledged=True)

In [45]:
# Check that the new restaurant was inserted
establishments.find_one({"name": "Penang Flavours", 
                         })

{'_id': ObjectId('65fc860de2745ce6ea87e1fb'), 'name': 'Penang Flavours'}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [46]:
##***# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = db.establishments.find({"BusinessType": "Restaurant/Cafe/Canteen"},
                               {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0})
for result in query:
    print(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'BusinessType': 'Restaur

3. Update the new restaurant with the `BusinessTypeID` you found.

In [47]:
# Update the new restaurant with the correct BusinessTypeID


In [48]:
# Confirm that the new restaurant was updated


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [49]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
print("Number of documents in result:", establishments.count_documents(query))

Number of documents in result: 0


In [50]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}
establishments.delete_many(query)
results = establishments.find(query)
for result in results:
    print(result)

In [51]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': 'Dover'}
print("Number of documents in result:", establishments.count_documents(query))

Number of documents in result: 0


In [52]:
# Check that other documents remain with 'find_one'
query = establishments.find_one({"LocalAuthorityName": "Dover"})


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [55]:
# Change the data type from String to Decimal for longitude and latitude
db.establishments.update_many({}, [{$set: {"geocode.longitude": 
                                           {$toDouble: "$geocode.longitude"},
                                           "geocode.latitude": 
                                           {$toDouble: "$geocode.latitude"}}}])



SyntaxError: invalid syntax (1915759436.py, line 2)

Use `update_many` to convert `RatingValue` to integer numbers.

In [56]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [57]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [
    {'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}
])

UpdateResult({'n': 38787, 'nModified': 34696, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [58]:
# Check that the coordinates and rating value are now numbers
print("Data types after conversion:")
print("Coordinates:", establishments['coordinates'].dtype)
print("RatingValue:", ['RatingValue'].dtype)

Data types after conversion:
Coordinates: Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments.coordinates.dtype')
RatingValue: Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments.RatingValue.dtype')
